In [23]:
# Import dependencies
# Import dependencies
import pandas as pd
import os
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
#from tensorflow.keras.callbacks import ModelCheckpoint
from sqlalchemy.orm import Session
from sqlalchemy import create_engine
from config import pw_postgres

In [4]:
# Import and read data
DB_PASS = pw_postgres
aws_database = 'shrbfk-final-project.cuitpsvagrne.us-east-2.rds.amazonaws.com:5432'
db_string = "postgresql+psycopg2://postgres:" + DB_PASS + "@" + aws_database + "/vet_data"
try:
    db_engine = create_engine(db_string)
    
except Exception as e:
    print(f"\nFailed to create database connection to {aws_database}.\n", e)
    exit()
    
combined_df = pd.read_sql_table('people_vet_join', db_engine)

In [6]:
combined_df.head()

,FIPS,State,County,PopChangeRate1819,PopChangeRate1019,TotalPopEst2019,NetMigrationRate1019,NaturalChangeRate1019,Net_International_Migration_Rate_2010_2019,PopChangeRate0010,...,WhiteVetsPct,BlackVetsPct,HispanicVetsPct,OtherRaceVetsPct,LessThanHSVetsPct,HighSchOnlyVetsPct,SomeCollegeVetsPct,CollegeDegreeVetsPct,EmployeedVetsPct,UnemployeedVetsPct
0,1000,AL,Alabama,0.317,2.461,4903185,1.059,1.402,0.809,7.48,...,72.534198,22.945607,2.022065,3.059596,6.396034,28.257189,37.385370,27.961408,71.330519,4.319786
1,1001,AL,Autauga,0.605,2.001,55869,0.686,1.315,-0.029,24.96,...,84.028832,9.673748,5.576631,1.119120,3.007812,20.859375,31.621094,44.511719,86.549340,3.558460
2,1003,AL,Baldwin,2.469,21.911,223234,21.001,0.910,0.714,29.80,...,90.621980,5.360321,1.439251,2.929360,3.612916,27.659792,36.676731,32.050560,74.927707,3.430532
3,1005,AL,Barbour,-0.748,-9.664,24686,-8.797,-0.867,0.161,-5.44,...,59.563253,37.349398,1.054217,3.087349,12.349398,39.834337,36.746988,11.069277,49.752066,9.966777
4,1007,AL,Bibb,0.121,-2.081,22394,-2.099,0.017,0.525,10.03,...,79.134682,18.492673,2.372645,0.000000,6.901510,54.708843,22.286125,16.103523,56.901408,0.000000


In [7]:
df_labels = combined_df[['State', 'County']]
combined_df = combined_df.drop(columns=['FIPS', 'State', 'County'])

In [8]:
combined_df = combined_df.drop([0,68,102,118,194,253,318,331,333,401,561,567,612,715,808,908,1014,
               1135,1200,1217,1242,1257,1341,1429,1512,1628,1685,1779,1797,1808,1830,
               1864,1927,2028,2082,2171,2249,2286,2354,2360,2407,2474,2570,2825,
               2855,2870,3005,3045,3101,3174])

In [9]:
combined_df.isnull()

,PopChangeRate1819,PopChangeRate1019,TotalPopEst2019,NetMigrationRate1019,NaturalChangeRate1019,Net_International_Migration_Rate_2010_2019,PopChangeRate0010,NetMigrationRate0010,NaturalChangeRate0010,Immigration_Rate_2000_2010,...,WhiteVetsPct,BlackVetsPct,HispanicVetsPct,OtherRaceVetsPct,LessThanHSVetsPct,HighSchOnlyVetsPct,SomeCollegeVetsPct,CollegeDegreeVetsPct,EmployeedVetsPct,UnemployeedVetsPct
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
5,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3295,True,False,False,True,True,True,False,True,True,True,...,False,False,False,False,False,False,False,False,False,False
3296,True,False,False,True,True,True,False,True,True,True,...,False,False,False,False,False,False,False,False,False,False
3297,True,False,False,True,True,True,False,True,True,True,...,False,False,False,False,False,False,False,False,False,False
3298,True,False,False,True,True,True,False,True,True,True,...,False,False,False,False,False,False,False,False,False,False


In [11]:
combined_df.dropna(inplace=True)

In [17]:
# Look at value counts for binning if needed
column_count= combined_df.TotalVets.value_counts()
column_count

496.0     6
211.0     5
552.0     5
626.0     5
667.0     5
         ..
6051.0    1
4136.0    1
6126.0    1
2202.0    1
3896.0    1
Name: TotalVets, Length: 2419, dtype: int64

In [ ]:
# Visualize the value count of column_count to identify which values to replace if needed
column_counts.plot.density()

In [30]:
# Split into features and target arrays
y = combined_df['TotalVets'].values
X = combined_df.drop(['TotalVets'],1).values

# Split preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split( X, y, random_state=42)

In [31]:
X.shape
X.reshape(-1,1)

array([[6.05000000e-01],
       [2.00100000e+00],
       [5.58690000e+04],
       ...,
       [1.70250896e+01],
       [7.15328467e+01],
       [1.02040816e+01]])

In [32]:
y.shape
y.reshape(-1,1)

array([[ 5272.],
       [19663.],
       [ 1328.],
       ...,
       [ 1069.],
       [  719.],
       [  576.]])

In [33]:
# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [34]:
# Fitting Random Forest Regression to the dataset
regressor = RandomForestRegressor(n_estimators = 10, random_state = 0)
regressor.fit(X,y)

RandomForestRegressor(n_estimators=10, random_state=0)

In [15]:
lasso = Lasso()
lasso.fit(X_train, y_train)
y_pred_lasso = lasso.predict(X_test)
mse = mean_squared_error(y_test, y_pred_lasso)
print(mse)

747319750.4261811


C:\Users\stace\anaconda3\envs\PythonData\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:532: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 220573500267.0246, tolerance: 875221503.1124347
  positive)


In [16]:
# Fitting Random Forest Regression to the dataset
regressor = RandomForestRegressor(n_estimators = 10, random_state = 0)
regressor.fit(X.reshape(-1,1), y.reshape(-1, 1))

ValueError: Found input variables with inconsistent numbers of samples: [310373, 3073]